![](../images/itam_logo.png)

M. Sc. Liliana Millán Núñez liliana.millan@itam.mx 

Octubre 2020

### Regresión Lineal

En Python, el paquete que nos permite hacer *machine learning* se llama `scikit-learn`. 

Para instalarlo en tu ambiente virtual de la clase necesitas poner `pip install sklearn`, esto te instalará tanto `scikit-learn`, como la librería `numpy`, la cual nos permite interactuar con arreglos y matrices de manera eficiente.

Puedes encontrar la API de referencia de `scikit-learn` [aquí](https://scikit-learn.org/stable/modules/classes.html).


#### Crash course sklearn 

Para todos los algoritmos en `scikit-learn` se tienen que acomodar los datos en formas muy específicas. Las variables explicativas $[X]$ y la variables *target* $[y]$ pueden estar en una lista, un arreglo de `numpy` o en un `DataFrame` de Pandas.  

##### Transformadores y Estimadores

En `scikit-learn` hay objetos **Estimadores** y **Transformadores**. Los transformadores, como su nombre indica, transforman los datos $(X)$ de alguna manera; los estimadores, realizan alguna estimación/predicción utilizando los datos $(X)$.

Ambos objetos tienen el método `fit()` a través del cual entrenan algún algoritmo o aprenden datos.

Los **transformadores** tienen el método `transform()` a través del cuál, aplican una transformación a los datos y los regresan transformados.

Los **estimadores** tienen el método `predict()` a través del cual, predicen un nuevo *output*  utilizando los datos $(X)$.

*** 

Ocuparemos `scikit-learn` para generar nuestro primer modelo, una regresión lineal.

Ocuparemos datos de mujeres en la India que tienen [diabetes](https://www.dropbox.com/s/wnynz7b8u9rnlqz/diabetes.csv?dl=0). 

Contamos con las siguientes variables: 

+ **Pregnancies:** Número de embarazos que la paciente ha tenido
+ **Glucose:** Nivel de glucosa en sangre
+ **BloodPressure:** Presión sanguínea
+ **SkinThickness:** Espesor de la piel (mm)
+ **Insulin:** Nivel de insulina en sangre
+ **BMI:** *Body Mass Index* Índice de masa coroporal
+ **DiabetesPedigreeFunction:** Función que extrapola si la paciente tendrá diabetes de acuerdo a su historial familiar 
+ **Age:** Edad de la paciente 
+ **Outcome:** Tiene diabetes (1) o no (0)

Queremos generar un modelo que nos permita predecir el nivel de insulina en la sangre dado el resto de las variables.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
np.random.seed(20201007)

In [ ]:
diabetes = pd.read_csv("../../data/diabetes_india/diabetes.csv")

In [ ]:
diabetes.shape

In [ ]:
diabetes.head()

In [ ]:
diabetes.describe()

In [ ]:
# cambiamos los nombres de columnas
diabetes.rename(columns={col: col.lower() for col in diabetes.columns.values}, inplace=True)

In [ ]:
diabetes.head()

In [ ]:
# cuántas observaciones hay que tienen blood_pressure en 0
diabetes.bloodpressure[diabetes.bloodpressure == 0].count()

In [ ]:
# cuántas observaciones hay que tienen blood_pressure en 0
diabetes.skinthickness[diabetes.skinthickness == 0].count()

In [ ]:
diabetes.insulin[diabetes.insulin == 0].count()

In [ ]:
diabetes.outcome.value_counts()

In [ ]:
sns.boxplot(diabetes.bmi)

In [ ]:
sns.boxplot(diabetes.insulin)

#### Validaciones

+ Validación 1: ok

In [ ]:
diabetes.head()

* Validación 2:

In [ ]:
sns.pairplot(data=diabetes, diag_kind="hist")

In [ ]:
X = diabetes.drop(['insulin'], axis=1)
y = diabetes.insulin

In [ ]:
X.shape

In [ ]:
type(X)

In [ ]:
X.head()

In [ ]:
y.shape

In [ ]:
type(y)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

In [ ]:
print(X_train.shape, y_train.shape)

In [ ]:
print(X_test.shape, y_test.shape)

#### Training (Hold out validation)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
# configuramos nuestra regresión lineal
rl = LinearRegression(fit_intercept=True)

In [ ]:
type(rl)

In [ ]:
m1 = rl.fit(X_train, y_train)

In [ ]:
m1

In [ ]:
m1.coef_

In [ ]:
X.columns

In [ ]:
feature_importances = pd.DataFrame({'betas': m1.coef_.tolist(),
                                   'features': X.columns.values})

In [ ]:
feature_importances.sort_values(by="betas", ascending=False)

In [ ]:
m1.intercept_

#### Predicciones

In [ ]:
predictions = m1.predict(X_test)

In [ ]:
predictions.shape

In [ ]:
predictions[:10]

#### Métricas de desempeño

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
mae = mean_absolute_error(y_test, predictions)
mae

In [ ]:
rmse = mean_squared_error(y_test, predictions, squared=False)
rmse

In [ ]:
mse = mean_squared_error(y_test, predictions)
mse

#### Validación 3 

* Residuales 

In [ ]:
residuals = y_test - predictions

In [ ]:
residuals.shape

In [ ]:
residuals_df = pd.DataFrame({'residuals': residuals, 
                            'predictions': predictions})

In [ ]:
residuals_df.head()

#### Q-Q plot

Tal vez necesites instalar el paquete `scipy` con `pip install scipy` en tu pyenv así como el paquete `pylab` con `pip install pylab`.

In [ ]:
import scipy.stats as stats
import pylab 

In [ ]:
stats.probplot(residuals_df.residuals, dist="norm", plot=pylab)
pylab.show()

Claramente no cumple con esta validación... pero solo para no dejar

In [ ]:
sns.scatterplot(x="predictions", y="residuals", data=residuals_df)